In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn import cross_validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
import xgboost

C:\Users\Dell\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.chdir(r"C:\Users\Dell\Desktop\Finalproject")

In [ ]:
df = pd.read_csv("XYZCorp_LendingData.txt",delimiter="\t",low_memory=False)

In [3]:
df1=pd.read_csv("pre_split.csv")

In [ ]:
df.isnull().sum()

In [ ]:
df1=df[df.revol_util<101]

In [ ]:
#the current loans are also added in the non defaulter list, which is not a good data to get insights from
#hence we are removing those rows 
df1 = df[df.out_prncp == 0] # 2.5 lcs

In [ ]:
#removing all the working data 
df1 = df1.drop(["addr_state","last_credit_pull_d","title","il_util", "max_bal_bc","total_bal_il","collection_recovery_fee", "collections_12_mths_ex_med", "desc", "funded_amnt", "funded_amnt_inv", "earliest_cr_line", "id", "member_id", "grade", "pymnt_plan", "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d", "out_prncp", "out_prncp_inv","emp_title", "zip_code", "recoveries", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "mths_since_rcnt_il", "inq_last_6mths","initial_list_status","total_acc"],axis=1)

In [ ]:
df1.info()

In [ ]:
#dropping any coulumns with more than 50% na values as the dataset is too big to go for 75% rate
df1 = df1.dropna(thresh=0.5*len(df1),axis=1)

In [ ]:
df1.info()

In [ ]:
df1.revol_util[df1.revol_util>100].count()

In [ ]:
#checking categorical columns with less than 1 unique value 
for col in df1.columns:
    if (len(df1[col].unique()) < 3):
        print(df1[col].value_counts())
        print()

In [ ]:
#removing policy code, for a single unique value
#removing application_type, due to the distribution being to baised
df1 = df1.drop(["policy_code","application_type"],axis=1)

In [ ]:
###checking the null values

In [ ]:
#checking and imputing missing values
df1.isnull().sum()

In [ ]:
#all the missing data columns are numerical
#columns with missing data
#total_rev_hi_lim,tot_cur_bal,tot_coll_amt,revol_util

In [4]:
def outliers(x):
    m = x.mean()
    s = np.std(x)
    lower_c = m-(3*s)
    upper_c = m+(3*s)
    n_upper = (x>upper_c).sum()
    n_lower = (x<lower_c).sum()
    val =  [lower_c, upper_c,n_upper,n_lower]
    return(val)

In [5]:
outliers(df1.revol_util)

[-20.031813215764025, 128.6548811939808, 5, 0]

In [6]:
outliers(df1.total_rev_hi_lim)

[-46882.66541043566, 106299.30258941099, 3814, 0]

In [7]:
outliers(df1.tot_coll_amt)

[-54547.18467391232, 54954.74635189998, 26, 0]

In [8]:
outliers(df1.tot_cur_bal)

[-257228.58895897743, 533885.3560228222, 3989, 0]

In [9]:
df1.revol_util.describe()

count    253030.000000
mean         54.311534
std          24.781165
min           0.000000
25%          36.300000
50%          55.800000
75%          73.900000
max         892.300000
Name: revol_util, dtype: float64

In [10]:
#
a = np.array([df1.total_rev_hi_lim])
p = np.percentile(a,95) # return 50th percentile, e.g median.
print(p)

67700.0


In [11]:
#
a = np.array([df1.revol_util])
p = np.percentile(a,95) # return 50th percentile, e.g median.
print(p)

92.3


In [12]:
#
a = np.array([df1.total_rev_hi_lim])
p = np.percentile(a,95) # return 50th percentile, e.g median.
print(p)

67700.0


In [13]:
#
a = np.array([df1.tot_coll_amt])
p = np.percentile(a,95) # return 50th percentile, e.g median.
print(p)

229.0


In [14]:
#
a = np.array([df1.tot_cur_bal])
p = np.percentile(a,95) # return 50th percentile, e.g median.
print(p)

382581.6


In [15]:
df1.total_rev_hi_lim.describe()

count    2.530300e+05
mean     2.970832e+04
std      2.553038e+04
min      0.000000e+00
25%      1.610000e+04
50%      2.970832e+04
75%      3.070000e+04
max      2.013133e+06
Name: total_rev_hi_lim, dtype: float64

In [16]:
df1["total_rev_hi_lim"] = np.where(df1["total_rev_hi_lim"] >106299.30258941099, 67700.0,df1['total_rev_hi_lim'])

In [17]:
df1.total_rev_hi_lim.describe()

count    253030.000000
mean      28280.020202
std       17106.535908
min           0.000000
25%       16100.000000
50%       29708.318589
75%       30700.000000
max      106200.000000
Name: total_rev_hi_lim, dtype: float64

In [18]:
df1.revol_util.describe()

count    253030.000000
mean         54.311534
std          24.781165
min           0.000000
25%          36.300000
50%          55.800000
75%          73.900000
max         892.300000
Name: revol_util, dtype: float64

In [19]:
df1["revol_util"] = np.where(df1["revol_util"] >892, 92.3,df1['revol_util'])

In [20]:
df1.revol_util.describe()

count    253030.000000
mean         54.308372
std          24.725221
min           0.000000
25%          36.300000
50%          55.800000
75%          73.900000
max         153.000000
Name: revol_util, dtype: float64

In [21]:
df1.tot_coll_amt.describe()

count    2.530300e+05
mean     2.037808e+02
std      1.825036e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.037808e+02
max      9.152545e+06
Name: tot_coll_amt, dtype: float64

In [22]:
df1["tot_coll_amt"] = np.where(df1["tot_coll_amt"] >9.152545e+06, 229.0,df1['tot_coll_amt'])

In [23]:
df1.tot_coll_amt.describe()

count    2.530300e+05
mean     2.037808e+02
std      1.825036e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.037808e+02
max      9.152545e+06
Name: tot_coll_amt, dtype: float64

In [24]:
df1.tot_cur_bal.describe()

count    2.530300e+05
mean     1.383284e+05
std      1.318526e+05
min      0.000000e+00
25%      3.842825e+04
50%      1.383284e+05
75%      1.649185e+05
max      8.000078e+06
Name: tot_cur_bal, dtype: float64

In [25]:
df1["tot_cur_bal"] = np.where(df1["tot_cur_bal"] > 8.000078e+06,382581.6,df1['tot_cur_bal'])

In [26]:
df1.tot_cur_bal.describe()


count    2.530300e+05
mean     1.383284e+05
std      1.318526e+05
min      0.000000e+00
25%      3.842825e+04
50%      1.383284e+05
75%      1.649185e+05
max      8.000078e+06
Name: tot_cur_bal, dtype: float64

In [ ]:
df1.total_rev_hi_lim.fillna(df1.total_rev_hi_lim.mean(),inplace=True)

In [ ]:
df1.tot_coll_amt.fillna(df1.tot_coll_amt.mean(),inplace=True)

In [ ]:
df1.tot_cur_bal.fillna(df1.tot_cur_bal.mean(),inplace=True) 

In [ ]:
 df1.revol_util.fillna(df1.revol_util.mean(),inplace=True) 

In [ ]:
df1.to_csv("non_active_data.csv",index=False)

In [ ]:
#treatment of categorical data

In [ ]:
#ordinal columns 
df1.sub_grade.unique

In [ ]:
mapped = {
    "emp_length":{
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0

    },
    
}

In [ ]:
df1 = df1.replace(mapped)

In [ ]:
#ALL categorical columns
obj_cols = ["verification_status","purpose","term","home_ownership","sub_grade"]

In [ ]:
# for preprocessing the data converting category to numerical
from sklearn import preprocessing
le={}

for x in obj_cols:
    le[x]=preprocessing.LabelEncoder()
    
for x in obj_cols:
    df1[x]=le[x].fit_transform(df1.__getattr__(x))
df1.head()

In [ ]:
df1.to_csv("pre_split.csv",index=False)

In [ ]:
#splitting data in train and test

In [ ]:
#data is supposed to be split according to gives dates

In [ ]:
df1.issue_d.unique()

In [ ]:
#array for test data and using isin function
a = ['Dec-2015', 'Nov-2015', 'Oct-2015', 'Sep-2015',
       'Aug-2015', 'Jul-2015', 'Jun-2015']
test = df1.loc[df1['issue_d'].isin(a)]

In [ ]:
train = df1.loc[~(df1['issue_d'].isin(a))]

In [ ]:
train.info()

In [ ]:
#dropping the date column
train = train.drop(["issue_d"],axis=1)
test = test.drop(["issue_d"],axis=1)

In [ ]:
#splitting
y_train = np.array(train['default_ind'])  
y_test = np.array(test['default_ind'])
X_train = train.drop('default_ind', axis = 1)  
X_train = np.array(X_train)
X_test = test.drop('default_ind', axis = 1)
X_test = np.array(X_test)

In [ ]:
# normalizing data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

In [ ]:
scaler=StandardScaler()
scaler.fit(X_test)
X_test=scaler.transform(X_test)

In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
dfz = pd.DataFrame(X_train)

In [ ]:
train.columns

In [ ]:
#trying metrics package to find the pvalues

In [ ]:
est = sm.OLS(y_train, X_train)
est2 = est.fit()
print(est2.summary())

In [ ]:
len(train.columns)

In [ ]:
train = train.drop(["tot_coll_amt","revol_bal","acc_now_delinq"],axis=1)
test = test.drop(["tot_coll_amt","revol_bal","acc_now_delinq"],axis=1)

In [ ]:
#checking for multicollinearity 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
df3 = df1._get_numeric_data()
X = add_constant(df3)
pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

In [ ]:
df1 = df1.drop(["installment","int_rate"],axis=1)

In [ ]:
#checking vif again after removing installement 
df3 = df1._get_numeric_data()
X = add_constant(df3)
pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

In [ ]:
train = train.drop(["installment","int_rate"],axis=1)
test = test.drop(["installment","int_rate"],axis=1)

In [ ]:
train.to_csv("train_loan.csv",index=False)

In [ ]:
test.to_csv("test_loan.csv",index=False)